<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [4]:
#!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
!pip install gensim

     |████████████████████████████████| 24.0 MB 99.9 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 5.2 MB/s eta 0:00:011


In [5]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

Посмотрим на примере одного слова, что из себя представляет embedding

In [10]:
word = 'dog'
print(wv_embeddings[word].shape)

(200,)


In [38]:
wv_embeddings.most_similar(positive="dog", topn=5)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804807662964),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

In [5]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слово `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [12]:
# method most_simmilar
'''your code'''

top_five = []
for animal in wv_embeddings.most_similar(positive="dog", topn=5):
  top_five.append(animal[0])

if "cat" in top_five:
  print("Cat is in the top 5")
else:
  print("Cat is NOT in the top 5")

Cat is NOT in the top 5


In [13]:
# Ищем в первой сотне
lst = wv_embeddings.most_similar(positive="dog", topn=100)
for animal in lst:
  if "cat" == animal[0]:
    print(f"Cat is in the top {lst.index(animal) + 1}")
    break

Cat is in the top 26


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [16]:
# !pip install nltk

In [17]:
import numpy as np
import re
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

In [18]:
q = tokenizer.tokenize("hello, bro, what's up dude")

In [19]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    '''your code'''
    
    
    features = np.zeros(dim, dtype="float32")
    token_list = tokenizer.tokenize(question)
    for word in token_list:
        if word in embeddings:
            features += embeddings[word]
    return features / len(token_list) # Делим на количество слов в вопросе

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [20]:
'''your code'''
vec = question_to_vec('I love neural networks', embeddings=wv_embeddings, tokenizer=tokenizer)
print(f'Third component is {vec[2].round(2):.2f}')

# В беседке ребята говорили, что делили на кол-во совпадений слов в словаре. Тогда выходит -1.29. 
# Но в задании, если я правильно понял, просят на кол-во слов в самом вопросе делить. Поправьте, если я не прав.

Third component is -0.96


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Вроде очевидно, что максимум будет, когда список состоит из одного элемента: [47]
Т.к. в этом случае Hits = 1, а DCG = 0. 

Иначе, если добавить хотя бы еще один элемент, то Hits уменьшится (если добавить единицу, то к тому же увеличится и DCG), что нас конечно не устраивает.

In [63]:
lst = [47]
print(f'max(Hits@47 - DCG@1) = {hits_count(lst, 47)} - {dcg_score(lst, 1)} = {hits_count(lst, 47) - dcg_score(lst, 1)}')

max(Hits@47 - DCG@1) = 1.0 - 0.0 = 1.0


<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
print(f'DCG@10 = {dcg_score([9], 10):.1f}')

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [56]:
import math

In [57]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    if len(dup_ranks) == 0:
        print("dup_ranks is emty")
        return -1
        
    hits_value = 0
    for rnk in dup_ranks:
        if rnk <= k:
            hits_value += 1
    return hits_value / len(dup_ranks)

In [64]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    if len(dup_ranks) == 0:
        print("dup_ranks is emty")
        return -1
    
    dcg_value = 0
    for rnk in dup_ranks:
        if rnk <= k:
            dcg_value += 1 / math.log2(1 + rnk)
    return dcg_value / len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [68]:
import pandas as pd

In [69]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
# Не совсем понял, как это написать в одну строчку, если еще итерироваться по copy_answers... 
# Поэтому взял просто первый элемент.
dup_ranks = [st.index(copy_answers[0]) + 1 for st in candidates_ranking if copy_answers[0] in st]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [70]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [71]:
!unzip stackoverflow_similar_questions.zip

unzip:  cannot find or open stackoverflow_similar_questions.zip, stackoverflow_similar_questions.zip.zip or stackoverflow_similar_questions.zip.ZIP.


Считайте данные.

In [25]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        '''your code'''
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [310]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [77]:
df = pd.DataFrame(validation_data)
df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,1000
0,How to print a binary heap tree without recurs...,How do you best convert a recursive function t...,How can i use ng-model with directive in angul...,flash: drawing and erasing,toggle react component using hide show classname,Use a usercontrol from another project to curr...,~ Paths resolved differently after upgrading t...,Materialize datepicker - Rendering when an ico...,Creating PyPi package - Could not find a versi...,How can I analyze a confusion matrix?,...,JRuby 1.7.0 will not install bundler given ple...,solr multiple tokenizers for query,Is it possible to pass an EJS variable to an A...,How To Read Data using Ip Address,SSIS XML Decryption Message,Detecting selection change on a UITextView?,Different CSS background-image depending on mo...,What's the equivalent averageifs for standard ...,Segmentation faults using f2py,Eclipse: Most useful refactorings\n
1,How to start PhoneStateListener programmatically?,PhoneStateListener and service,Java cast object[] to model,WCF and What does this mean?,How to uncheck checkbox using jQuery Uniform l...,Two projects with same code base,Can't read php file when upload image to serve...,create pandas dataframe from dictionary of dic...,how to pass tkinter text to flex,runtime error project source code not found an...,...,setGraphic() not working correctly on recursiv...,Strange behaviour of sscanf with string,HTTP method for a WCF Restful Service,SubSonic .Filter() in memory filter,Does ASP.NET MVC use the regular toolbox contr...,How can I render a GSP as a String?,publishing ASP.Net MVC 4 Project,how to hyperlink to a TFS item from visual studio,mutex attribute PTHREAD_PROCESS_SHARED inverts...,Unable to call SOAP Webservice using jQuery\n


Размер нескольких первых строк

In [78]:
for i in range(5):
    print(i + 1, len(validation_data[0]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [88]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [264]:
from numpy import indices

In [293]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    
    q_vec = question_to_vec(question, embeddings, tokenizer).reshape(1, -1)
    c_vec = [question_to_vec(cndt, embeddings, tokenizer) for cndt in candidates]
    lst_cos_distance = cosine_similarity(q_vec, c_vec)[0]
    idx = np.argsort(-lst_cos_distance)
    
    return [list(enumerate(candidates))[i] for i in idx]

In [276]:
res = rank_candidates('converting string to list', ['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'], wv_embeddings, tokenizer)
print(res)

[(2, 'How to use jQuery AJAX for an outside domain?'), (0, 'Convert Google results object (pure js) to Python object'), (1, 'C# create cookie from string and send it')]


Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [294]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',       # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [295]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [296]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(1, 'Getting all list items of an unordered list in PHP'), #скрыт
            (0, 'select2 not displaying search results'), #скрыт
            (2, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ.** Во втором эксперименте: **102**

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [297]:
from tqdm.notebook import tqdm

In [332]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [333]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.276 | Hits@   1: 0.276
DCG@   5: 0.332 | Hits@   5: 0.385
DCG@  10: 0.349 | Hits@  10: 0.438
DCG@ 100: 0.395 | Hits@ 100: 0.671
DCG@ 500: 0.421 | Hits@ 500: 0.874
DCG@1000: 0.434 | Hits@1000: 1.000



### Эмбеддинги, обученные на корпусе похожих вопросов

In [26]:
train_data = read_corpus('./data/train.tsv')

In [27]:
train_data[:5]

[['converting string to list',
  'Convert Google results object (pure js) to Python object\n'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
  'Event handling for geometries in Three.js?\n'],
 ['Sending array via Ajax fails',
  'Getting all list items of an unordered list in PHP\n'],
 ['How to insert CookieCollection to CookieContainer?',
  'C# create cookie from string and send it\n'],
 ['Updating one element of a bound Observable collection',
  'WPF- How to update the changes in list item of a list\n']]

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор. 

**Ответ.** Я выбрал размер окна=6, т.к. при данном наборе получалось наилучнее качество для топ1 и топ5. Мне показалось это важнее, чем равномерное увеличение (200, 300, 500, 800, 1000) т.к. нет смысла в топ500, скажем. Поэтому перебором нашел наилучшее качество для топ1 и топ5.

In [28]:
'''your code'''
words = [word[0] + ' ' + word[1] for word in train_data]

In [29]:
words[0]

'converting string to list Convert Google results object (pure js) to Python object\n'

In [32]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=6).wv

In [362]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [363]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.405 | Hits@   1: 0.405
DCG@   5: 0.413 | Hits@   5: 0.419
DCG@  10: 0.415 | Hits@  10: 0.425
DCG@ 100: 0.430 | Hits@ 100: 0.507
DCG@ 500: 0.457 | Hits@ 500: 0.724
DCG@1000: 0.486 | Hits@1000: 1.000



### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:
На самом деле, я не до конца понял, почему у нас фукнция ранжирования так устроена. Почему туда посылается по одному вопросу, и лист с вопросами для ранжирования. Почему бы туда не подавать сразу два листа. Она же быдет куда быстрее работать, чем в цикле это все делать. Все равно я там где-то беру [0] т.к. эта функция возвращает лист листов. А там нужен только первый элемент, потому что вопрос один всего. В общем, я не понял глубинного смысла ахах.

Про токенайзеры: 

WordPunctTokenizer() получился самый норм из всей троицы (WordPunctTokenizer, re, word_tokenize). По-видимому, просто регулярка \w+|[^\w\s]+ в данном случае лучше справляется на таком паке вопросов и к тому же пропускает знаки пунктуации и не выделяет их в отдельные токены.

Нормализация помогает, но совсем немного. Точнее помогает лемматизация.

Почему качество такое плохое: 

Мне кажется, тут много факторов играет роль:
1. Вообще сам BOW мне не сильно нравится. По сути, усреднять слова и получить из этого представления вектора целого предложения как-то не очень. 
2. Также количество данных, на котором мы обучаем не очень большая. Возможно стоит увелить размер датасета и обучать модель на бо'льших данных.
3. Тематика валидационных данных отличается от тренировочных.

Как можно улучшить на мой взгляд:
1. Я бы попробовал через SVD и метрику TF-IDF. Тут я не совсем шарю, как работает word2vec внутри себя.
2. Попробовать другие эмбеддинги, например glove
3. Сделать токенизацию поумнее, не через nltk